A demonstration of ArcticPipeline using the Mnist dataset. 

In [2]:
!pip install pydicom
import arcticpipeline001 as ap
import torch
import torch.nn as nn
import torch.nn.functional as F



     |████████████████████████████████| 1.9MB 14.2MB/s eta 0:00:01     |████████████████████████▎       | 1.4MB 14.2MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:

class BASICCNN(nn.Module):
    
    def __init__(self, image_size=100, features=5):
        
        super(BASICCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
        
    def forward(self, x):
        
        # x is not your typical x, rather it is a dictionary of feature groups. Feature groups allow data to be transferred from the 
        # data frame to the learning network in discreate groupings, for example for networks that have multiple stages. 
        
        # The network should be written in a way that pulls in these seperate feature groups as needed. 
        # 
        x = x['mnist_image'][0]
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        
        return F.log_softmax(x)
            
    
gpu = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

state_model = BASICCNN().to(gpu)

In [4]:

mnist_arctic = ap.ArcticPipeline(batch_size=128, num_workers=0, pin_memory=False, shuffle=True)
mnist_arctic.data_mode = "train"



mnist_arctic.import_dataset("/storage/mnist-arctic/dataset/")

mnist_arctic.data_mode = "train"
mnist_arctic.image_field = "filename"


mnist_arctic.define_process_image_command_list([])


mnist_arctic.compose_data(bucket_list=['train', 'test'], 
                          command_list=[
    ('List Current Dataset',  mnist_arctic.show_state(n=5, active=False)),
                              
                          ])
    
# do not modify data after _init_learn, as it's too late!
                              
mnist_arctic._init_learn(y_label_column = 'label', 
                                                 network = state_model,
                                                 loss_function = nn.CrossEntropyLoss(),
                                                 feature_group_dict = {'mnist_image': ['_img_from_file']}
                                                      )
    
    


CUDA cpu
Setting the dataframe:  train to dataframe containing:  60000
Setting the dataframe:  test to dataframe containing:  10000
Setting the dataframe:  all to dataframe containing:  70000
Arctic_Pipeline.show_state() is set to inactive. To turn it on, use .show_state(active=True) in the compose method
Arctic_Pipeline.show_state() is set to inactive. To turn it on, use .show_state(active=True) in the compose method
ArcticPipeline @: /storage/mnist-arctic/dataset/
Bucket: train Len:60000
Bucket: test Len:10000
Bucket: val Len:0
Bucket: all Len:70000
 setting network to BASICCNN(
  (conv1): Conv2d(3, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)


In [5]:

mnist_arctic.data_mode = "train"
mnist_arctic.fit(epochs=10)
mnist_arctic.predict("all",1)


  0%|          | 0/469 [00:00<02:28,  3.16it/s]  

Epoch:   0, train BCE: 0.4982
Eval Mode


  0%|          | 1/469 [00:25<1:01:15,  7.85s/it]

              val BCE: 35.7090


  0%|          | 0/469 [00:00<01:43,  4.53it/s]  

...

              val BCE: 23.0754


  0%|          | 0/469 [00:00<01:44,  4.48it/s]  

Epoch:   9, train BCE: 0.1740
Eval Mode


  0%|          | 0/469 [00:17<01:44,  4.48it/s]

              val BCE: 19.7928
Predict Mode: all
Actual Value: tensor([1, 8, 1, 9, 3, 5, 0, 8, 1, 7, 4, 4, 7, 0, 3, 3, 7, 8, 3, 4, 8, 1, 5, 3,
        8, 3, 4, 1, 1, 8, 8, 4, 1, 1, 2, 3, 3, 4, 0, 5, 7, 0, 0, 3, 3, 5, 9, 1,
        6, 1, 4, 6, 6, 8, 8, 1, 6, 1, 1, 8, 8, 9, 6, 5, 5, 9, 5, 7, 9, 9, 5, 2,
        8, 4, 5, 0, 5, 5, 7, 9, 0, 4, 1, 8, 9, 5, 1, 6, 7, 8, 9, 6, 5, 2, 4, 9,
        2, 7, 3, 6, 3, 9, 3, 3, 1, 3, 1, 8, 7, 3, 2, 6, 5, 8, 4, 1, 0, 6, 1, 1,
        4, 4, 2, 5, 5, 9, 2, 6]) Prediction: tensor([1, 8, 1, 9, 3, 5, 0, 8, 1, 7, 4, 4, 7, 0, 3, 3, 7, 8, 3, 4, 8, 1, 5, 3,
        8, 3, 4, 1, 1, 8, 8, 4, 1, 1, 2, 3, 3, 4, 0, 5, 7, 0, 0, 3, 3, 5, 4, 1,
        6, 1, 4, 6, 6, 8, 8, 1, 6, 1, 1, 8, 8, 9, 6, 5, 5, 9, 5, 7, 9, 9, 5, 2,
        8, 4, 5, 0, 5, 5, 7, 9, 0, 4, 1, 8, 9, 5, 1, 6, 7, 8, 9, 6, 5, 2, 4, 9,
        2, 7, 3, 6, 3, 9, 3, 3, 1, 3, 1, 8, 7, 3, 2, 6, 5, 8, 4, 1, 0, 6, 1, 1,
        4, 4, 2, 5, 5, 9, 2, 6])
